# Atlas split

Split 3d or 4d atlas images into individual volumes to feed into LEAD DBS connectome_mapper as seeds

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
import numpy as np
import nibabel as nib

def split_atlas_into_rois(atlas, atlas_labels):
    """
    Split atlas into 3d images for each label (works for both 3D and 4D atlases)

    Parameters:
    - atlas: nibabel.nifti1.Nifti1Image, 3D or 4D atlas
    - atlas_labels: List of labels

    Returns:
    - dict: dict of arrays with ROI masks
    """
    atlas_data = atlas.get_fdata()
    atlas_shape = atlas_data.shape

    # Check the dimensionality of the atlas
    is_4d = len(atlas_shape) == 4

    if is_4d:
        x, y, z, t = atlas_shape
    else:
        x, y, z = atlas_shape

    if not is_4d: assert len(atlas_labels) + 1 == len(np.unique(atlas_data)), "atlas_labels and unique values in atlas must have the same length"

    roi_dict = {}

    if is_4d:
        
        for t_idx in range(t):

            # Extract the values from the atlas
            roi_image = atlas_data[:, :, :, t_idx]
            
            roi_image[roi_image > 0] = 1

            # Add the ROI image to the dictionary
            key = atlas_labels[t_idx]
            roi_dict[key] = roi_image
    
    else:

        for i, roi in enumerate(atlas_labels):

            # Extract the values from the atlas
            roi_image = atlas_data * (atlas_data == i + 1)

            roi_image[roi_image > 0] = 1

            # Add the ROI image to the dictionary
            key = roi
            roi_dict[key] = roi_image

    return roi_dict


# Schaefer400x7 atlas

In [3]:
schaefer_atlas_path = "../input/atlases/Schaefer400Parcels7NetworksPlusSubcort.nii.gz"
schaefer_labels_path = "../input/atlases/Schaefer400Parcels7NetworksPlusSubcort_labels.txt"

schaefer_atlas = nib.load(schaefer_atlas_path)
schaefer_labels = pd.read_csv(schaefer_labels_path, sep="\t", header=None).values.tolist()[0]

In [5]:
schaefer_atlas_dict = split_atlas_into_rois(schaefer_atlas, schaefer_labels)

In [7]:
output_dir_schaefer = "../output/atlas_rois/schaefer400_plus_subcort/"
for key in schaefer_atlas_dict.keys():
    atlas = nib.Nifti1Image(schaefer_atlas_dict[key], schaefer_atlas.affine)
    nib.save(atlas, output_dir_schaefer + key + ".nii.gz")

# Tian atlas

In [18]:
tian_atlas_path = "../input/atlases/Tian_Subcortex_S1_3T_2009cAsym_regrid.nii.gz"
tian_labels_path = "../input/atlases/Tian_Subcortex_S1_3T_label.txt"

tian_atlas = nib.load(tian_atlas_path)
tian_labels = pd.read_csv(tian_labels_path, sep="\t", header=None).values.flatten().tolist()

In [19]:
tian_atlas_dict = split_atlas_into_rois(tian_atlas, tian_labels)

In [20]:
output_dir_tian = "../output/atlas_rois/tian_subcortical_atlas/"
Path(output_dir_tian).mkdir(parents=True, exist_ok=True)
for key in tian_atlas_dict.keys():
    atlas = nib.Nifti1Image(tian_atlas_dict[key], tian_atlas.affine)
    nib.save(atlas, output_dir_tian + key + ".nii.gz")